## Credit to https://github.com/johnnyf-hcp/vault-demo-kmip-mysql

In [ ]:
%env WORKDIR=/tmp/vault
%env VAULT_KMIP_PORT=5696
%env KMIP_SVC_NAME=my-service
%env KMIP_ROLE_NAME=admin
%env MYSQL_PORT=3306
%env MYSQL_TAG=latest
%env ORACLE_REG_EMAIL=jose.merchan@hashicorp.com
%env ORACLE_REG_PASSWORD="'#np@sy^QL7NCdJ"

In [ ]:
%%bash
cat $WORKDIR/VAULT_TOKEN.TXT
cat $WORKDIR/VAULT_ADDR.TXT

In [ ]:
%env VAULT_ADDR=https://vault-primary-europe-southwest1-rche.josemerchan-f986bf.gcp.sbx.hashicorpdemo.com:8200
%env VAULT_TOKEN=hvs.TOQpapl9gf4vE5R68194BK6k

In [ ]:
%%bash
# Log into the Oracle docker registry
echo "$ORACLE_REG_PASSWORD" | docker login container-registry.oracle.com -u $ORACLE_REG_EMAIL --password-stdin
# Pull the MySQL Enterprise docker image
docker pull container-registry.oracle.com/mysql/enterprise-server:latest

# Setting up HashiCorp Vault

In [ ]:
%%bash
# Enable KMIP secrets engine at the default mount path
#vault secrets disable kmip
vault secrets enable kmip

# Start KMIP Server and set client TLS certificate TTL to 365 days

# Get the docker IP address of the Vault server to configure the certificate SANs.  Otherwise the MongoDB KMIP connection will fail.
export KMIP_ADDR=$(terraform  output -state=../1_Platform_Deployment/terraform.tfstate -raw cluster_primary_fqdn_kmip)
export FQDN=$(echo "$KMIP_ADDR" | sed -E 's~https://([^:/]+).*~\1~')
export IP=$(dig $FQDN +short)


# Option 1 - Elliptic Curve (EC) is the default
vault write kmip/config listen_addrs=0.0.0.0 default_tls_client_ttl=365d server_hostnames=$FQDN,localhost server_ips=$IP,127.0.0.1
# Option 2 - RSA
#vault write kmip/config listen_addrs=0.0.0.0:$VAULT_KMIP_PORT tls_ca_key_type="rsa" tls_ca_key_bits=2048 default_tls_client_ttl=365d server_hostnames=$VAULT_IP,localhost

# Check the KMIP configuration
echo
vault read kmip/config

## Step 2 - Creating the KMIP certificates for MySQL Server TDE

In [ ]:
# Create a folder for the KMIP certificates to be used by MySQL Server
! mkdir -p $WORKDIR/mysql-keyring-okv/ssl

In [ ]:
%%bash
# Read and store the CA certificate.  Note that in linux, the casing of the name needs to be CA.pem and not ca.pem.
# On MacOS, the casing doesn't matter.
vault read -format=json kmip/ca | jq -r .data.ca_pem > $WORKDIR/mysql-keyring-okv/ssl/CA.pem

echo "CA certification:"
cat $WORKDIR/mysql-keyring-okv/ssl/CA.pem

## Step 2a - Create KMIP Scope in Vault

The KMIP secrets engine uses the concept of scopes to partition KMIP managed object storage into multiple named buckets. Within a scope, roles can be created which dictate the set of allowed operations that the particular role can perform. TLS client certificates can be generated for a role, which services and applications can then use when sending KMIP requests against Vault's KMIP secret engine.

In order to generate client certificates for KMIP clients to interact with Vault's KMIP server, we must first create a scope and role and specify the desired set of allowed operations for it.

In [ ]:
%%bash
# Settings for the KMIP scope and role to be created
# Create a scope
vault write -f kmip/scope/$KMIP_SVC_NAME

# List scopes and verify the KMIP scope has been created
echo
vault list kmip/scope

In [ ]:
%%bash
# Create the KMIP role (note the TTL if not set, it uses the default Vault token/lease TTL of 768hrs)
vault write kmip/scope/$KMIP_SVC_NAME/role/$KMIP_ROLE_NAME operation_all=true tls_client_ttl=365d

# List roles and verify the KMIP role has been created
echo
vault list kmip/scope/$KMIP_SVC_NAME/role

In [ ]:
# Optional - View the role details
! vault read kmip/scope/$KMIP_SVC_NAME/role/$KMIP_ROLE_NAME

In [ ]:
%%bash
# Generate the required certificates
vault write -format=json \
  kmip/scope/$KMIP_SVC_NAME/role/$KMIP_ROLE_NAME/credential/generate \
  format=pem > $WORKDIR/credential.json

# Save the certificate and private key
jq -r .data.certificate < $WORKDIR/credential.json > $WORKDIR/mysql-keyring-okv/ssl/cert.pem
jq -r .data.private_key < $WORKDIR/credential.json > $WORKDIR/mysql-keyring-okv/ssl/key.pem

# Remove the temp file
rm $WORKDIR/credential.json

## Step 3 - Configuring MySQL Server for TDE with Vault as the KMIP server

In [ ]:
%%bash
# Get the docker IP address of the Vault server
export KMIP_ADDR1=$(terraform  output -state=../1_Platform_Deployment/terraform.tfstate -raw cluster_primary_fqdn_kmip)
export FQDN1=$(echo "$KMIP_ADDR1" | sed -E 's~https://([^:/]+).*~\1~')
export IP1=$(dig $FQDN1 +short)

export KMIP_ADDR2=$(terraform  output -state=../1_Platform_Deployment/terraform.tfstate -raw cluster_dr_fqdn_kmip)
export FQDN2=$(echo "$KMIP_ADDR2" | sed -E 's~https://([^:/]+).*~\1~')
export IP2=$(dig $FQDN2 +short)

# Create the okvclient.ora file.  Specifies the KMIP server for MySQL Server.
# For this demo, we will be using the same Vault server as the standby.
# Note: Do not create this file on a windows machines as it inserts a CRLF character instead of a LF character on the line break.
# This will cause an error on MySQL server on load.
echo
tee $WORKDIR/mysql-keyring-okv/okvclient.ora <<EOF
SERVER=$IP1:$VAULT_KMIP_PORT
STANDBY_SERVER=$IP2:$VAULT_KMIP_PORT
EOF

In [ ]:
%%bash
# Configure my.cnf that MySQL Server will use.
tee $WORKDIR/my.cnf <<EOF
[mysqld]
port=3306
early-plugin-load=keyring_okv.so
keyring_okv_conf_dir=/mysql-keyring-okv
EOF

In [ ]:
! kill $(lsof -t -i:3306)

In [ ]:
%%bash
# Run MySQL Enterprise in docker.
# Expose both the MySQL port to the host machine.

echo "MySQL Container Version Used: $MYSQL_TAG"
echo
docker run -d --rm --name mysql-enterprise \
--mount type=bind,src=$WORKDIR/my.cnf,dst=/etc/my.cnf \
--mount type=bind,src=$WORKDIR/mysql-keyring-okv,dst=/mysql-keyring-okv \
-p ${MYSQL_PORT}:${MYSQL_PORT} container-registry.oracle.com/mysql/enterprise-server:$MYSQL_TAG


In [ ]:
# Verify that the MySQL Server docker container is running
! docker ps

## Step 4 - MySQL Server TDE Configuration Steps

In [ ]:
# Run in a terminal in steps using comments as separator
alias mysql="docker exec -it mysql-enterprise mysql"
export MYSQL_GEN_ROOT_PASSWORD=$(docker logs mysql-enterprise 2>&1 | grep GENERATED | cut -d " " -f5)
export MYSQL_ROOT_PASSWORD=root
echo "Generated MySQL root password: $MYSQL_GEN_ROOT_PASSWORD"

mysql --connect-expired-password -u root -p"$MYSQL_GEN_ROOT_PASSWORD" -e "ALTER USER 'root'@'localhost' IDENTIFIED BY '$MYSQL_ROOT_PASSWORD';"

# Verify that the keyring_okv plugin is installed and the status is ACTIVE
mysql -u root -p'root' -e \
"SELECT PLUGIN_NAME, PLUGIN_STATUS
FROM INFORMATION_SCHEMA.PLUGINS
WHERE PLUGIN_NAME LIKE 'keyring%';"

# Install the general purpose keyring key-management functions
# https://dev.mysql.com/doc/refman/8.0/en/keyring-functions-general-purpose.html
mysql -u root -p'root' -e \
"INSTALL PLUGIN keyring_udf SONAME 'keyring_udf.so';
CREATE FUNCTION keyring_key_generate RETURNS INTEGER
SONAME 'keyring_udf.so';
CREATE FUNCTION keyring_key_fetch RETURNS STRING
SONAME 'keyring_udf.so';
CREATE FUNCTION keyring_key_length_fetch RETURNS INTEGER
SONAME 'keyring_udf.so';
CREATE FUNCTION keyring_key_type_fetch RETURNS STRING
SONAME 'keyring_udf.so';
CREATE FUNCTION keyring_key_store RETURNS INTEGER
SONAME 'keyring_udf.so';
CREATE FUNCTION keyring_key_remove RETURNS INTEGER
SONAME 'keyring_udf.so';"

## Step 5 - Testing MySQL TDE functionality

In [ ]:
# Create a new random key and store it on the keyring
# If the KMIP configuration is not done right, this function will return an error.
# Also note it will throw an error if the MyKey1 already exists
mysql -u root -p'root' -e \
"SELECT keyring_key_generate('MyKey1', 'AES', 32);"

In [ ]:
# Test creating a test database and table
mysql -u root -p'root' -e \
"CREATE DATABASE scripting;"

mysql -u root -p'root' -e \
"USE scripting;
CREATE TABLE test_encryption (
  ID int(10) unsigned NOT NULL AUTO_INCREMENT,
  server_name varchar(15) NOT NULL,
  PRIMARY KEY (id)
) ENGINE=InnoDB AUTO_INCREMENT=1 DEFAULT CHARSET=latin1 ENCRYPTION = 'Y';"


In [ ]:
# Run this command to insert test data into the table
mysql -u root -p'root' -e \
"USE scripting;
INSERT INTO test_encryption(server_name)
VALUES (CONCAT('websvr-', SUBSTR(MD5(RAND()),1,8)));"

# View the records in the table
mysql -u root -p'root' -e \
"USE scripting;
SELECT * FROM test_encryption;"

In [ ]:
# Test master key rotation
mysql -u root -p'root' -e \
"ALTER INSTANCE ROTATE INNODB MASTER KEY;"

In [ ]:
# Simulate that the KMIP server is down by changing the listening port to 5697
! vault write kmip/config listen_addrs=0.0.0.0:5697

In [ ]:
# Verify that I can still insert and select data
mysql -u root -p'root' -e \
"USE scripting;
INSERT INTO test_encryption(server_name)
VALUES (CONCAT('websvr-', SUBSTR(MD5(RAND()),1,8)));"

# View the records in the table
mysql -u root -p'root' -e \
"USE scripting;
SELECT * FROM test_encryption;"

In [ ]:
# Try rotating the master key.  As the KMIP server is "down", you should get an error.
mysql -u root -p'root' -e \
"ALTER INSTANCE ROTATE INNODB MASTER KEY;"

In [ ]:
%%bash
# Simulate restoring back the KMIP server by updating back to the original KMIP port number
export KMIP_ADDR=$(terraform  output -state=../1_Platform_Deployment/terraform.tfstate -raw cluster_primary_fqdn_kmip)
export FQDN=$(echo "$KMIP_ADDR" | sed -E 's~https://([^:/]+).*~\1~')
export IP=$(dig $FQDN +short)


# Option 1 - Elliptic Curve (EC) is the default
vault write kmip/config listen_addrs=0.0.0.0 default_tls_client_ttl=365d server_hostnames=$FQDN,localhost server_ips=$IP,127.0.0.1

In [ ]:
# Try rotating the master key again.  As the KMIP server is now "restored", this command should now return no errors.
mysql -u root -p'root' -e \
"ALTER INSTANCE ROTATE INNODB MASTER KEY;"

# Cleanup

In [ ]:
%%bash
# Cleanup

# Disable KMIP secrets engine
vault secrets disable kmip

# Remove KMIP certificate demo files
rm -rf $WORKDIR/mysql-keyring-okv

# Stop MySQL container
docker stop mysql-enterprise